In [1]:
from rdflib import Graph
from rdflib.term import URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD
import itertools
import os
from tqdm.auto import tqdm
from torch import nn

import torch
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig, AutoModelForCausalLM
from rag_reduce import ont_query_reduce
import gc

In [14]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_id = "google/gemma-2b-it"
# model_id = "microsoft/Phi-3-mini-128k-instruct"
# model_id = "unsloth/llama-3-8b-bnb-4bit"


tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
txt = gen_prompt(r1, r2, query, include_sample1=True, include_sample2=True)

messages = [
    # {"role": "system", "content": "You are an Ontology Alignment expert. You are able to align two ontologies by creating a file in EDOAL format containing the result alignments. You are able to produce complex alignments that are those involving multiple entities and relationships in a n:m cardinality. The user will provide you with two ontologies and you respond with the EDOAL file containing the alignments. You don't need to explain yourself. Just give as response the resulting file without saying anything."},
    {"role": "user", "content": txt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)

print(input_ids.shape)

torch.Size([1, 3325])


In [21]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "google/gemma-2b-it"
# model_id = "microsoft/Phi-3-mini-128k-instruct"
# model_id = "unsloth/llama-3-8b-bnb-4bit"


tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    quantization_config=quantization_config,
    low_cpu_mem_usage=True

)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
messages = [
    # {"role": "system", "content": "You are an Ontology Alignment expert. You are able to align two ontologies by creating a file in EDOAL format containing the result alignments. You are able to produce complex alignments that are those involving multiple entities and relationships in a n:m cardinality. The user will provide you with two ontologies and you respond with the EDOAL file containing the alignments. You don't need to explain yourself. Just give as response the resulting file without saying anything."},
    {"role": "user", "content": txt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

print(input_ids.shape)

torch.Size([1, 3325])


In [23]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=2 * 1024,
        eos_token_id=terminators,
        do_sample=False,
        temperature=None,
        top_p=None,

    )
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
         xmlns:align="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#"
         xmlns:edoal="http://ns.inria.org/edoal/1.0/#">
    <Alignment>
        <xml>yes</xml>
        <level>2EDOAL</level>
        <type>**</type>
        <onto1>
            <Ontology rdf:about="http://cmt#"/>
        </onto1>
        <onto2>
            <Ontology rdf:about="http://confOf#"/>
        </onto2>
        <map>
            <Cell>
                <entity1 rdf:resource="http://cmt#Administrator"/>
                <entity2>
                    <edoal:Class>
                        <edoal:And>
                            <edoal:Class rdf:about="http://confOf#Administrator"/>
                            <edoal:PropertyRestriction>
                               